## LOAD

In [1]:
%load_ext autotime
%load_ext autoreload
%autoreload 2
%matplotlib inline 
#inline
# %pdb
%config InlineBackend.figure_format ='retina'

import os
import pathlib
import sys

## setting path
## set the base path
# os.chdir(pathlib.Path(os.path.dirname(path[0])).parent.absolute())
# ## set path to sys for Lmbda functio
# sys.path.insert(0, os.getcwd())

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

time: 1.1 s (started: 2023-04-14 12:07:44 +02:00)


In [2]:
!zenml stack set ireland_green_deployment_stack

Connected to the ZenML server: 'http://127.0.0.1:8237'
Running with active workspace: 'default' (repository)
Active repository stack set to: 'ireland_green_deployment_stack'.
⠙ Setting the repository active stack to 'ireland_green_deployment_stack'...oyment_stack'...
time: 4.31 s (started: 2023-03-27 13:49:08 +02:00)


## IMPORT

In [3]:
import pandas as pd
pd.options.plotting.backend = "plotly"
# import dtale
# from prophet import Prophet
from neuralprophet import NeuralProphet, set_log_level, set_random_seed
set_log_level("ERROR")
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import t
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report, precision_recall_fscore_support
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

import mlflow
mlflow.set_tracking_uri("http://0.0.0.0:7009")
# from mlflow import MlflowClient 

import dtale

# from lightgbm import LGBMRegressor
# import dtale
from statsmodels.tsa.seasonal import STL
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import power_transform, scale

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# from sklearn.inspection import DecisionBoundaryDisplay
from imblearn.ensemble import RUSBoostClassifier, EasyEnsembleClassifier, BalancedBaggingClassifier, BalancedRandomForestClassifier
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, accuracy_score
from sklearn.metrics import explained_variance_score, max_error, r2_score, mean_absolute_error, f1_score

from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from imblearn.pipeline import Pipeline
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsRegressor, NeighborhoodComponentsAnalysis
# from lightgbm import LGBMClassifier
# import lightgbm as lgb
from sklearn.linear_model import *
from sklearn.ensemble import VotingRegressor, GradientBoostingRegressor, RandomForestRegressor, VotingClassifier
from sklearn.cross_decomposition import PLSRegression
from sklearn.neighbors import KNeighborsRegressor

from imblearn.over_sampling import SMOTE
from sklearn.calibration import CalibratedClassifierCV
import plotly.graph_objects as go
from statsmodels.miscmodels.ordinal_model import OrderedModel

from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import OneHotEncoder
import itertools
import warnings

warnings.filterwarnings('ignore')

np.random.seed(0)


from absl import logging as absl_logging
import warnings

warnings.filterwarnings("ignore")
absl_logging.set_verbosity(-10000)

import mlflow
from zenml.steps import Output, step


import sqlalchemy
import yaml
from typing import Dict, Union
from copy import deepcopy

from sklearn.base import ClassifierMixin
from sklearn.pipeline import Pipeline

import feast
from feast import FeatureStore

from evidently.report import Report
from evidently.metric_preset import (
    DataQualityPreset,
    DataDriftPreset,
    ClassificationPreset,
)

from evidently.report import Report
from evidently.metrics.base_metric import generate_column_metrics
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset
from evidently.metrics import *

from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset, DataQualityTestPreset
from evidently.tests import *

from evidently.metric_preset import (
    DataQualityPreset,
    DataDriftPreset,
    ClassificationPreset,
)
from evidently.report import Report
from zenml.integrations.mlflow.flavors.mlflow_experiment_tracker_flavor import (
    MLFlowExperimentTrackerSettings,
)


WARNING - (py.warnings._showwarnmsg) - /opt/conda/conda/envs/azba/lib/python3.9/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



distutils Version classes are deprecated. Use packaging.version instead.




time: 4.25 s (started: 2023-03-27 13:49:12 +02:00)


## FUNCTIONS

In [21]:
def hourlySample(data):
    d = data.copy()
    d.index = pd.DatetimeIndex(d.index) 
    return d.reindex(pd.date_range(start=d.index.min(), periods = 24 * len(d), freq='H'), method='ffill').squeeze()

def findOutliers(df):
    s = df.copy()
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    iqr_lower = q1 - 8 * iqr
    iqr_upper = q3 + 8 * iqr
#     outliers = s[(s < iqr_lower) | (s > iqr_upper)]
    s[s < iqr_lower] = iqr_lower
    s[s > iqr_upper] = iqr_upper
    
    return s

def _scores(pred, true):
    display(confusion_matrix(true.loc[pred.index], pred))
    print("bAcc : ", bscore := balanced_accuracy_score(true.loc[pred.index], pred))
    print("acc :" , acc := accuracy_score(true.loc[pred.index], pred))
    print("f1 :", f1 := f1_score(true.loc[pred.index], pred))
    print("report :", classification_report(true.loc[pred.index], pred))
    print("precision, recall", prScore := precision_recall_fscore_support(true.loc[pred.index], pred, average='weighted'))
    
    return (bscore, acc, f1, prScore)

def _scoresArr(pred, true, printVal=True):
    bscore = balanced_accuracy_score(true, pred)
    acc = accuracy_score(true, pred)
    f1 = f1_score(true, pred)
    prScore = precision_recall_fscore_support(true, pred, average='weighted')
        
    if printVal:
        display(confusion_matrix(true, pred))
        print("bAcc : ", bscore)
        print("acc :" , acc)
        print("f1 :", f1)
        print("report :", classification_report(true, pred))
        print("precision, recall", )
    
    return (bscore, acc, f1, prScore)

def _scoresRegArr(actual, pred, printVal=True):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
        
    if printVal:
        print("rmse : ", rmse)
        print("mae : ", mae)
        print("r2 : ", r2)
        
    return (rmse, mae, r2)
    
def _toMultiIndex(d, name=None):
    reformed_dict = {}
    for outerKey, innerDict in d.items():
        for innerKey, values in innerDict.items():
            reformed_dict[(outerKey,
                           innerKey)] = values

    l = pd.DataFrame(reformed_dict)
    display(l)
    if name:
        l.to_csv(name + ".csv")
        
    return l

time: 67.2 ms (started: 2023-03-27 13:58:26 +02:00)


## DATA

### Fetch

In [5]:
@step
def getDates() -> Output(
    stDate = str, endDate=str
):
    stDate = '2022-07-01'
    endDate = '2023-02-21'
    
    return stDate, endDate

@step
def fetchData(_stDate:str, _endDate:str)-> Output(dataFrame=pd.DataFrame):

    dd = [{'epoch_time' : int(pd.Timestamp(d.strftime('%Y-%m-%d %H:%M:%S')).timestamp())} for d in pd.date_range(start=_stDate, end=_endDate, freq='H')]
    
    repoPath = "<path of repo>"
    store = FeatureStore(repo_path=repoPath)

    batch_features = store.get_online_features(
        entity_rows=dd,
        features=store.get_feature_service("ireland_data_service", allow_cache=True),
    ).to_df()
    
    df = batch_features.copy()
    
    df.set_index('epoch_time', inplace=True)
    df.index = pd.to_datetime(df.index, unit='s')
    df.index.name = None
    
    dataFrame = df.astype(float)
    
    # drop the first NaN value as that might be because of shift
    dataFrame = dataFrame.iloc[dataFrame.notnull().all(axis=1).argmax() :]
    dataFrame = dataFrame.groupby(by=dataFrame.index, as_index=True).mean()
    
    dataFrame = dataFrame.reindex(
        pd.date_range(
            start=dataFrame.index.min(), end=dataFrame.index.max(), freq="H"
        ),
        fill_value=np.nan,
    )
    dataFrame = dataFrame.interpolate(method="bfill", axis=0)
    
    return dataFrame

time: 36 ms (started: 2023-03-27 13:49:17 +02:00)


### Filter

In [6]:
@step
def filteredData(
    dataConcatenated: pd.DataFrame, _stDate: str
) -> Output(targetSpreadWOOutlier=pd.DataFrame, features=pd.DataFrame):
    data = dataConcatenated.loc[_stDate:][:-1]
    
    targetSpread = data["price_diff"].dropna().asfreq("h")
    features = data.loc[
        targetSpread.index, ~data.columns.isin(["price_diff", "ie_actual_wnd"])
    ].asfreq("h")

    targetSpreadWOOutlier = findOutliers(targetSpread).to_frame()
    
    features = features.loc[: , ['day', 'weekday', 'year_day', 'week', 'month', 'quarter', 'lag1',
       'lag2', 'lag3', 'lag4', 'lag5', 'lag6', 'lag7', 'lag30']]
    
    return targetSpreadWOOutlier, features

time: 33.8 ms (started: 2023-03-27 13:49:17 +02:00)


### Prepare

In [18]:
@step
def prepareFeatures(features : pd.DataFrame, _stDate:str, _endDate:str)-> Output(
        X_test=pd.DataFrame, X_day_test=pd.DataFrame
    ):
    
    features = features.dropna()
    dayFeatures = features.resample('D', axis=0).sum() 
    
    featureColumns = list(features.columns)
    
    X_test = features.loc[_stDate : _endDate, featureColumns]
    X_day_test = features.loc[_stDate : _endDate, featureColumns].resample('D', axis=0).sum()
    
    return X_test, X_day_test

time: 171 ms (started: 2023-03-27 13:56:28 +02:00)


### Save

In [8]:
@step
def saveDataToCsv(preds : list, trues: list)->pd.DataFrame:
    pred1 = pd.DataFrame(preds[0], index=trues[0].index)
    pred2 = hourlySample(pd.DataFrame(preds[1], index=trues[1]))
    
    df = pd.concat([pred1, pred2], axis=1)
    df.to_csv('predictions.csv')
    
    return df

time: 33.3 ms (started: 2023-03-27 13:49:17 +02:00)


## MODEL

In [10]:
from zenml.services import BaseService
from zenml.client import Client

@step(enable_cache=False)
def prediction_service_loader() -> Output(
    trendService=BaseService, dayService=BaseService
):
    """Load the model service of our train_evaluate_deploy_pipeline."""
    client = Client()
    model_deployer = client.active_stack.model_deployer
    trendServices = model_deployer.find_model_server(
        pipeline_name="ireland_eval",
        pipeline_step_name="trendModelDeployer",
        running=True,
    )
    
    dayServices = model_deployer.find_model_server(
        pipeline_name="ireland_eval",
        pipeline_step_name="dayModelDeployer",
        running=True,
    )
    
    trendService = trendServices[0]
    dayService = dayServices[0]
    
    return trendService, dayService

@step
def predictor(
    trendService: BaseService,
    dayService: BaseService,
    data: pd.DataFrame,
    dayData: pd.DataFrame,
) -> Output(predictions=list, true=list):
    """Run a inference request against a prediction service"""
    
    trendService.start(timeout=100)  # should be a NOP if already started
    trendPrediction = trendService.predict(data.to_numpy())

    dayService.start(timeout=100)  # should be a NOP if already started
    dayPrediction = dayService.predict(dayData.to_numpy())

    predictions = [trendPrediction, dayPrediction]
    true = [data, dayData]

    return predictions, true
    


time: 60.6 ms (started: 2023-03-27 13:49:17 +02:00)


## REPORT

In [23]:
@step
def checkDataQuality(data:pd.DataFrame) -> pd.DataFrame:
    
    data_quality_report = TestSuite(tests=[
        TestNumberOfColumnsWithMissingValues(),
        TestNumberOfRowsWithMissingValues(),
        TestNumberOfConstantColumns(),
        TestNumberOfDuplicatedRows(),
        TestNumberOfDuplicatedColumns(),
        DataQualityTestPreset(),
#         DataStabilityTestPreset()
    ])
    data.rename(columns={"price_diff": "target"}, inplace=True)
    data_quality_report.run(reference_data=None, current_data=data)
    data_quality_report.save_html("data_quality_report.html")
    
    return data
    

@step(
    experiment_tracker="ireland_green_tracker",
    settings={
        "experiment_tracker.mlflow": MLFlowExperimentTrackerSettings(
            experiment_name="ireland_eval",
            nested=True,
            tags={"mlflow.runName": "inference_reports"},
        )
    },
)
def publishResultAndReport(
    target: pd.Series, features: pd.DataFrame, preds: list, trues: list
) -> pd.DataFrame:

    
    targetSign = np.sign(target.mask(target <= 0, -1))
    index = trues[0].index
    graphs = []
    
    # -----      Trend Model -------------------------------
    predTrend = pd.DataFrame(preds[0], index=index, columns=["prediction"])
    
    pnL = (predTrend.squeeze() * target.loc[predTrend.index].squeeze())
    graphs.append(go.Scatter(x=predTrend.index, 
               y=pnL.cumsum(),
               mode='lines',
               name="trend"
            )
    )
    
    finalReportDFTrend = pd.concat(
        [
            targetSign.to_frame("target").loc[index, :],
            predTrend,
            features.loc[index, :],
        ],
        axis=1,
    )
    
    reportTrend = Report(
        metrics=[ClassificationPreset()]
    )
    reportTrend.run(
        reference_data=finalReportDFTrend.loc[
            finalReportDFTrend.index < "2022-12-27", :
        ],
        current_data=finalReportDFTrend.loc[
            finalReportDFTrend.index >= "2022-12-27", :
        ],
    )

    mlflow.log_dict(predTrend.to_dict("records"), "results/trend/predictions.json")
    mlflow.log_dict(
        finalReportDFTrend.loc[finalReportDFTrend.index < "2022-12-27", :].to_dict(
            "records"
        ),
        "results/trend/reference_data.json",
    )
    mlflow.log_dict(
        finalReportDFTrend.loc[finalReportDFTrend.index >= "2022-12-27", :].to_dict(
            "records"
        ),
        "results/trend/current_data.json",
    )
    mlflow.log_dict(reportTrend.as_dict(), "reports/trend/prediction_report.yml")
    reportTrend.save_html("trend_prediction_report.html")
    mlflow.log_artifact(
        "trend_prediction_report.html",
        "reports/trend",
    )
    # -----      DAY Model -------------------------------
    
    predDay = hourlySample(
        pd.DataFrame(preds[1], index=trues[1].index, columns=["prediction"])
    )
    
    pnL = (predDay.squeeze() * target.loc[predDay.index].squeeze())
    graphs.append(go.Scatter(x=predDay.index, 
               y=pnL.cumsum(),
               mode='lines',
               name="day"
            )
    )
    
    finalReportDFDay = pd.concat(
        [
            targetSign.to_frame("target").loc[index, :],
            predDay,
            features.loc[index, :],
        ],
        axis=1,
    )
    reportDay = Report(
        metrics=[DataQualityPreset(), DataDriftPreset(), ClassificationPreset()]
    )
    reportDay.run(
        reference_data=finalReportDFDay.loc[finalReportDFDay.index < "2022-12-27", :],
        current_data=finalReportDFDay.loc[finalReportDFDay.index >= "2022-12-27", :],
    )

    mlflow.log_dict(predDay.to_frame().to_dict("records"), "results/day/predictions.json")
    mlflow.log_dict(
        finalReportDFDay.loc[finalReportDFDay.index < "2022-12-27", :].to_dict(
            "records"
        ),
        "results/day/reference_data.json",
    )
    mlflow.log_dict(
        finalReportDFDay.loc[finalReportDFDay.index >= "2022-12-27", :].to_dict(
            "records"
        ),
        "results/day/current_data.json",
    )
    mlflow.log_dict(reportDay.as_dict(), "reports/day/prediction_report.yml")
    reportTrend.save_html("day_prediction_report.html")
    mlflow.log_artifact(
        "day_prediction_report.html",
        "reports/day",
    )

    df = pd.concat(
        [
            predTrend.rename(columns={"prediction": "trend_prediction"}),
            predDay.to_frame().rename(columns={"prediction": "day_prediction"}),
        ],
        axis=1,
    )
    
    fig = go.Figure(data = graphs)
    fig.update_layout(
        autosize=False,
        width=1700,
        height=500,
#         legend=dict(groupclick="toggleitem")
    )
    
    mlflow.log_dict(df.to_dict("records"), "predictions.json")
    mlflow.log_figure(fig, f'pnl.html')
    
    return df


time: 191 ms (started: 2023-03-27 14:03:28 +02:00)


## PIPELINE

In [24]:
from zenml.pipelines import pipeline

@pipeline(
    enable_cache=False, name="ireland_eval"
)
def inference_pipeline(
    dates,
    importer,
    qualityChecker,
    filter_data,
    prepare,
    prediction_service_loader,
    predictor,
    publisher
):
    """Basic inference pipeline."""
    
    (_stDate, _endDate) = dates()
    data = importer(_stDate, _endDate)
    qualityData = qualityChecker(data)
    
    target, features = filter_data(data, _stDate)
    
    X_test, X_day_test = prepare(features, _stDate, _endDate)
    
    trendService, dayService = prediction_service_loader()
    predictions, trues = predictor(trendService, dayService, X_test, X_day_test)
    
    _ = publisher(target, features, predictions, trues)
    

# Initialize an inference pipeline run
my_inference_pipeline = inference_pipeline(
    dates = getDates(),
    qualityChecker = checkDataQuality(),
    importer = fetchData(),
    filter_data= filteredData(), 
    prepare = prepareFeatures(),
    prediction_service_loader=prediction_service_loader(),
    predictor=predictor(),
    publisher=publishResultAndReport()
)

my_inference_pipeline.run()

Registered pipeline inference_pipeline (version 4).
Running pipeline inference_pipeline on stack ireland_green_deployment_stack (caching disabled)
Step dates has started.
Step dates has finished in 0.846s.
Step prediction_service_loader has started.
Step prediction_service_loader has finished in 0.988s.
Step importer has started.
Step importer has finished in 4.536s.
Step qualityChecker has started.
Step qualityChecker has finished in 6.411s.
Step filter_data has started.
Step filter_data has finished in 2.115s.
Step prepare has started.
Step prepare has finished in 2.568s.
Step predictor has started.


Step predictor has finished in 2.954s.
Step publisher has started.
Step publisher has finished in 37.802s.
Pipeline run inference_pipeline-2023_03_27-12_03_31_928466 has finished in 1m8s.
Dashboard URL: http://127.0.0.1:8237/workspaces/default/pipelines/7242b106-7cfb-4c2b-8a83-b26be488ebdd/runs
time: 1min 10s (started: 2023-03-27 14:03:29 +02:00)
